# Intro
Hi kagglers, I was inspired to write this article by the outstanding works of <a href='https://www.kaggle.com/code/kelde9/darts-ensemble-stores-sales-forecasting'>Tom Keldenich</a> and <a href="https://www.kaggle.com/code/ferdinandberr/darts-forecasting-deep-learning-global-models">Ferdinand Berr</a>. Thanks to them, I picked up interesting ideas, their development gave a good score: 0.3804.

The key difference lies in the technique of combining model predictions. While he employed blending in work, I opted for a different approach - stacking. Also, I used another type of encoding and hyperparameters for the models.

First of all, I will focus on a detailed description of the used approach. If you need more information about EDA and data preparation, I suggest reading the works of others authors.

They comprehensively described these steps, so there is no need to dwell on them again.

But first things first.Before using  Darts with stacking, I tried other techniques and I'm happy to share the results. Perhaps this will help save time by avoiding going in the wrong direction.

<b>Here some score of them</b>:
- Prophet 1782 timeseries with exogenus seasonality by years and covariates: 0.59676
- SARIMAX 1782 timeseries with exogenus seasonality by years: 0.46409
- as a previous plus target variable log1p transformation : 0.49116
- hybrid models: SARIMAX prediction for main data and LightGBM training on residuals:  0.48831
- CatBoost 54 timeseries grouped by families: 0.53593
- LightGBM 54 timeseries grouped by families: 0.47366
- as a previous plus target variable log1p transformation : 0.42178

Darts:
- LightGBM + One hot + Blending 4 lags: 0.38127
- LightGBM + Ordinal: 0.3839
- LightGBM + Ordinal + without Scaler: 0.38697
- LightGBM + Ordinal+ Blending 4 lags: 0.38107
- LightGBM + Ordinal + Blending 6 lags: 0.38055
- LightGBM + Ordinal + Blending 6 lags + Robust Scaler: 0.39995

## 时间序列预测相关概念

​	时间序列预测是一种通过历史数据预测未来值的方法，通常用于具有时间顺序的数据，如股市价格、气温变化、销售额等。在时间序列预测中，关键概念包括：

​	1.**时间序列数据**：数据点按时间顺序排列，通常以等时间间隔收集，例如每天、每小时、每月等。

​	2.**自回归模型 (AR)**：一种基于过去的观测值预测未来的方法。假设未来值可以通过过去的值（滞后值）来预测。

​	3.**滑动窗口 (Rolling window)**：当使用过去的数据进行预测时，通常只选择一个固定的时间窗口。例如，使用过去30天的数据来预测接下来一天的值。

​	4.**未来协变量 (Future Covariates)**：也称为外生变量，是指影响目标变量的额外信息，但这些信息在预测时是已知的。比如，假设你预测明天的销售量，未来协变量可能包括广告预算或节假日等信息。

​	5.**过去协变量 (Past Covariates)**：指与目标变量相关的历史数据，这些数据有助于模型理解时间序列的趋势和季节性变化。例如，历史销售数据或天气数据。

时间序列预测的主要目标是识别数据中的模式（如趋势、季节性等），并利用这些模式来预测未来。

## Setup

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
!pip install darts &> /dev/null

In [3]:
import darts
print(darts.__version__)

0.23.1


In [4]:
import warnings
warnings.filterwarnings('ignore')

# Import Data
Our objective is to forecast 16 days sales of 54 stores in Ecuador based on data from the period of January 01, 2013, to August 15, 2017.

### 目标：
- **预测任务**是要预测54家商店在未来16天的销售数据。

### 数据范围：
- **训练数据时间范围**：数据的时间跨度从**2013年1月1日**到**2017年8月15日**，这是用来训练模型的数据时间范围。

### train

In [5]:
import pandas as pd
import numpy as np

In [6]:
df_train = pd.read_csv('/home/jibing/KaggleCode/StoreSales/data/train.csv')
display(df_train.head())

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


Please note that we do not have sales data for December 25 of each year. We will need to fill in the missing data:

In [7]:
df_train.iloc[np.where(df_train['date'].str.contains('12-25'))]

,id,date,store_nbr,family,sales,onpromotion


###  holidays_events

In [8]:
df_holidays_events = pd.read_csv('/home/jibing/KaggleCode/StoreSales/data/holidays_events.csv')
display(df_holidays_events.head())

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [9]:
df_holidays_events.groupby(['type', 'locale'])['locale_name'].count()

type        locale  
Additional  Local        11
            National     40
Bridge      National      5
Event       National     56
Holiday     Local       137
            National     60
            Regional     24
Transfer    Local         4
            National      8
Work Day    National      5
Name: locale_name, dtype: int64

### oil

In [10]:
df_oil = pd.read_csv('/home/jibing/KaggleCode/StoreSales/data/oil.csv')
display(df_oil.head())

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


### store

In [11]:
df_stores = pd.read_csv('/home/jibing/KaggleCode/StoreSales/data/stores.csv')
display(df_stores.head())

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


### transactions

In [12]:
df_transactions = pd.read_csv('/home/jibing/KaggleCode/StoreSales/data/transactions.csv')
display(df_transactions.head())

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


Note: a transaction is a receipt created after a customer’s purchase

交易指的是 **顾客购买商品后生成的收据**

###  test and sample_submission

In [13]:
df_test = pd.read_csv('/home/jibing/KaggleCode/StoreSales/data/test.csv')
df_sample_submission = pd.read_csv('/home/jibing/KaggleCode/StoreSales/data/sample_submission.csv')
display(df_test.head())
display(df_sample_submission.head())

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,0.0
4,3000892,0.0


# Preprocessing

In [14]:
family_list = df_train['family'].unique()
store_list = df_stores['store_nbr'].unique()
display(family_list)
display(store_list)

array(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
       'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
       'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
       'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
       'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
       'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
       'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
       'SEAFOOD'], dtype=object)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54])

Let's combine the training and test datasets. So we can find covariates and extract IDs for prediction on the test set.

In [15]:
train_merged = pd.merge(df_train, df_stores, on ='store_nbr')
train_merged = train_merged.sort_values(["store_nbr","family","date"])
train_merged = train_merged.astype({"store_nbr":'str', "family":'str', "city":'str',
                          "state":'str', "type":'str', "cluster":'str'})

display(train_merged.head())

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13
1782,1782,2013-01-02,1,AUTOMOTIVE,2.0,0,Quito,Pichincha,D,13
3564,3564,2013-01-03,1,AUTOMOTIVE,3.0,0,Quito,Pichincha,D,13
5346,5346,2013-01-04,1,AUTOMOTIVE,3.0,0,Quito,Pichincha,D,13
7128,7128,2013-01-05,1,AUTOMOTIVE,5.0,0,Quito,Pichincha,D,13


#### 上述代码完成了训练数据和商店数据的合并、排序和数据类型转换。通过合并这些数据，最终得到了一个包含了销售、商店信息以及商品类别的完整数据集。

## TimeSeries

In [16]:
from darts import TimeSeries
from tqdm import tqdm

In [17]:
family_TS_dict = {}

for family in tqdm(family_list):
    df_family = train_merged.loc[train_merged['family'] == family]

    list_of_TS_family = TimeSeries.from_group_dataframe(
                                df_family,
                                time_col="date",
                                group_cols=["store_nbr","family"], # columns for grouping time series
                                static_cols=["city","state","type","cluster"], # static covariates
                                value_cols="sales", # target
                                fill_missing_dates=True, # filling missing dates, remember Dec 25th
                                freq='D' # days
                                )
    for ts in list_of_TS_family:
            ts = ts.astype(np.float32)

    list_of_TS_family = sorted(list_of_TS_family, key=lambda ts: int(ts.static_covariates_values()[0,0]))
    family_TS_dict[family] = list_of_TS_family

100%|██████████| 33/33 [00:16<00:00,  2.03it/s]


Let's talk more about TimeSeries. This is the main object of the Darts library. Understanding its specifics will greatly simplify the comprehension of the subsequent code. We have created a dictionary with Multivariate TimeSeries
Dictionary structure:

In [18]:
family_TS_dict.keys()

dict_keys(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS', 'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS', 'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE', 'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES', 'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE', 'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE', 'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY', 'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES', 'SEAFOOD'])

Timeseries for family 'AUTOMOTIVE' and store_nbr 1
date: number (range) of days in the time series

component: data from the columns that we passed in the parameter 'value_cols'. In our case, the number of daily sales.

In [19]:
family_TS_dict['AUTOMOTIVE'][0]

<TimeSeries (DataArray) (date: 1688, component: 1, sample: 1)> Size: 14kB
array([[[0.]],

       [[2.]],

       [[3.]],

       ...,

       [[1.]],

       [[1.]],

       [[4.]]])
Coordinates:
  * date       (date) datetime64[ns] 14kB 2013-01-01 2013-01-02 ... 2017-08-15
  * component  (component) object 8B 'sales'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  static_covariates store_nbr      family   city      s...
    hierarchy:          None

Constant values for all observations in the group:

In [20]:
family_TS_dict['AUTOMOTIVE'][0].static_covariates

static_covariates,store_nbr,family,city,state,type,cluster
component,,,,,,
sales,1,AUTOMOTIVE,Quito,Pichincha,D,13


We can select specific dates with slicing:

In [21]:
family_TS_dict['AUTOMOTIVE'][0][:10]

<TimeSeries (DataArray) (date: 10, component: 1, sample: 1)> Size: 80B
array([[[0.]],

       [[2.]],

       [[3.]],

       [[3.]],

       [[5.]],

       [[2.]],

       [[0.]],

       [[2.]],

       [[2.]],

       [[2.]]])
Coordinates:
  * date       (date) datetime64[ns] 80B 2013-01-01 2013-01-02 ... 2013-01-10
  * component  (component) object 8B 'sales'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  static_covariates store_nbr      family   city      s...
    hierarchy:          None

#### 上述代码将每个商品类别的销售数据转换为时间序列格式，并按商店进行分组

### Normalizing time series
Let's create a pipeline to automate our actions.
We will use the following data preprocessing:
- Filling missing values.
- Encoding of static covariates. In my example, this is Ordinal Encoding. You can use any other one from the sklearn. It must have methods: fit(), transform() , inverse_transform().
- Target variable transformation. log1p significantly improves prediction accuracy.
- Scaler. Default is Min Max Scaler.

In [22]:
from darts.dataprocessing import Pipeline
from darts.dataprocessing.transformers import Scaler, StaticCovariatesTransformer, MissingValuesFiller, InvertibleMapper
from sklearn.preprocessing import OrdinalEncoder

In [23]:
family_pipeline_dict = {}
family_TS_transformed_dict = {}   # 经过预处理后的时间序列数据

for key in family_TS_dict:
    train_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Fill NAs")
    static_cov_transformer = StaticCovariatesTransformer(verbose=False, transformer_cat = OrdinalEncoder(), name="Encoder")
    log_transformer = InvertibleMapper(np.log1p, np.expm1, verbose=False, n_jobs=-1, name="Log-Transform")   
    train_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaling")

    train_pipeline = Pipeline([train_filler,
                             static_cov_transformer,
                             log_transformer,
                             train_scaler])

    training_transformed = train_pipeline.fit_transform(family_TS_dict[key])
    family_pipeline_dict[key] = train_pipeline
    family_TS_transformed_dict[key] = training_transformed

- **填补缺失值**：对于时间序列数据，尤其是存在缺失日期（例如12月25日没有销售数据）的情况，填补缺失值是非常重要的。这样可以保证时间序列数据的连续性，不会影响模型的训练。
- **静态协变量编码**：商品的静态特征（如商店类型、城市、州等）需要被编码成数字格式，`OrdinalEncoder` 可以将这些类别变量转化为数值。
- **目标变量转换**：使用 `log1p` 转换有助于处理数据中的极端值，使得数据分布更加平滑，从而改善模型的预测性能。
- **数据标准化**：归一化处理使得不同特征的数值在同一尺度上，防止某些特征因为取值范围较大而对模型产生过大影响。

In [24]:
family_TS_transformed_dict['AUTOMOTIVE'][0][:10]

<TimeSeries (DataArray) (date: 10, component: 1, sample: 1)> Size: 80B
array([[[0.        ]],

       [[0.36672579]],

       [[0.46275643]],

       [[0.46275643]],

       [[0.598104  ]],

       [[0.36672579]],

       [[0.        ]],

       [[0.36672579]],

       [[0.36672579]],

       [[0.36672579]]])
Coordinates:
  * date       (date) datetime64[ns] 80B 2013-01-01 2013-01-02 ... 2013-01-10
  * component  (component) object 8B 'sales'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  static_covariates  store_nbr  family  city  state  ty...
    hierarchy:          None

## Covariates

> **A covariate** is a variable that helps to predict a target variable.

This covariate can be dependent on the target variable. For example, the type of store, `type`, where the sales are made. But it can also be independent. For example, the price of oil on the day of the sale of a product.

This covariate can be known in advance, for example in our dataset we have the price of oil from January 1, 2013 to August 31, 2017. In this case, we talk about a **future covariate**.

There are also **past covariates**. These are covariates that are not known in advance. For example in our dataset, the transactions are known for the dates January 1, 2013 to August 15, 2017.

## 依赖协变量 (Dependent Covariates)：
这些协变量与目标变量有关联。例如，销售发生时的商店类型（type）就是一个依赖协变量，因为商店类型可能会影响销售额。

## 独立协变量 (Independent Covariates)：
这些协变量与目标变量没有直接的依赖关系。例如，产品销售当天的油价就是一个独立协变量，因为油价可能影响销售，但它不是由销售直接决定的。

## **未来协变量（Future Covariates）**：

- 这些协变量是可以在预测过程中已知的。例如，在我们的数据集中，油价从2013年1月1日到2017年8月31日都是已知的。因此，油价是一个未来协变量，因为它是预测目标变量（销售量）时可以提前获得的信息。

未来协变量通常用于时间序列预测模型中的外部输入数据，尤其是在已知未来的情况下，可以提高预测精度。

## **过去协变量（Past Covariates）**：

- 这些协变量是随着时间的推移逐步了解的，因此无法提前知晓。在我们的数据集中，交易量就是一个过去协变量。交易数据从2013年1月1日到2017年8月15日已经存在，作为已知信息，可以用来预测未来的销售额。

过去协变量通常用于反映过去的事件对当前目标变量的影响，并且常常是时间序列模型中的重要输入。

### Date （未来协变量）

下述代码通过生成和处理时间特征为模型提供了丰富的上下文信息。具体步骤包括：

- 从日期中提取多个时间特征（如年份、月份、星期等）。
- 创建一个线性增长的时间序列作为时间特征。
- 将所有时间特征堆叠为一个多维时间序列。
- 对这些时间特征进行标准化处理，以便于后续建模。

In [25]:
from darts.utils.timeseries_generation import datetime_attribute_timeseries

In [26]:
full_time_period = pd.date_range(start='2013-01-01', end='2017-08-31', freq='D')


year = datetime_attribute_timeseries(time_index = full_time_period, attribute="year")
month = datetime_attribute_timeseries(time_index = full_time_period, attribute="month")
day = datetime_attribute_timeseries(time_index = full_time_period, attribute="day")
dayofyear = datetime_attribute_timeseries(time_index = full_time_period, attribute="dayofyear")
weekday = datetime_attribute_timeseries(time_index = full_time_period, attribute="dayofweek")
weekofyear = datetime_attribute_timeseries(time_index = full_time_period, attribute="weekofyear")
timesteps = TimeSeries.from_times_and_values(times=full_time_period,
                                             values=np.arange(len(full_time_period)),
                                             columns=["linear_increase"])

time_cov = year.stack(month).stack(day).stack(dayofyear).stack(weekday).stack(weekofyear).stack(timesteps)
time_cov = time_cov.astype(np.float32)

In [27]:
display(print(time_cov.components.values))
display(time_cov[100])

['year' 'month' 'day' 'dayofyear' 'dayofweek' 'weekofyear'
 'linear_increase']


None

<TimeSeries (DataArray) (time: 1, component: 7, sample: 1)> Size: 28B
array([[[2013.],
        [   4.],
        [  11.],
        [ 101.],
        [   3.],
        [  15.],
        [ 100.]]], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 8B 2013-04-11
  * component  (component) object 56B 'year' 'month' ... 'linear_increase'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [28]:
time_cov_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")
time_cov_train, time_cov_val = time_cov.split_before(pd.Timestamp('20170816'))
time_cov_scaler.fit(time_cov_train)
time_cov_transformed = time_cov_scaler.transform(time_cov)

### Oil （未来协变量）

- **数据加载**：首先从油价数据表 `df_oil` 中加载油价时间序列数据。

- **缺失值处理和标准化**：对油价数据进行缺失值填充和标准化处理，确保数据完整且数值处于统一的尺度。

- **计算移动平均**：通过7天和28天的移动平均滤波器，提取油价的短期和长期趋势。

- **特征堆叠**：将两个移动平均时间序列堆叠到一起，创建包含多个特征的多维时间序列，作为模型输入的额外协变量。

In [29]:
from darts.models.filtering.moving_average_filter import MovingAverageFilter

移动平均特征的计算有以下核心作用：

平滑短期波动，减少噪声对模型的干扰。
提取不同时间尺度上的趋势，丰富模型输入的时间特征。
识别潜在的趋势拐点，帮助预测转折点。
满足实际业务中对短期或长期趋势的分析需求。

In [30]:
# Oil Price

oil = TimeSeries.from_dataframe(df_oil, 
                                time_col = 'date', 
                                value_cols = ['dcoilwtico'],
                                freq = 'D')

oil = oil.astype(np.float32)

# Transform
oil_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Filler")
oil_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")
oil_pipeline = Pipeline([oil_filler, oil_scaler])
oil_transformed = oil_pipeline.fit_transform(oil)

# Moving Averages for Oil Price
oil_moving_average_7 = MovingAverageFilter(window=7)
oil_moving_average_28 = MovingAverageFilter(window=28)

oil_moving_averages = []

ma_7 = oil_moving_average_7.filter(oil_transformed).astype(np.float32)
ma_7 = ma_7.with_columns_renamed(col_names=ma_7.components, col_names_new="oil_ma_7")
ma_28 = oil_moving_average_28.filter(oil_transformed).astype(np.float32)
ma_28 = ma_28.with_columns_renamed(col_names=ma_28.components, col_names_new="oil_ma_28")
oil_moving_averages = ma_7.stack(ma_28)

In [31]:
display(oil_moving_averages[100])

<TimeSeries (DataArray) (date: 1, component: 2, sample: 1)> Size: 8B
array([[[0.78416604],
        [0.78240633]]], dtype=float32)
Coordinates:
  * date       (date) datetime64[ns] 8B 2013-04-11
  * component  (component) object 16B 'oil_ma_7' 'oil_ma_28'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

### Holidays  （未来协变量）

## 这段代码的功能是为商店生成假期特征，具体包括：

按日期记录国家级假期、地方假期、特殊事件（地震、足球等）和其他节日类型。

为每个商店创建一个包含这些假期特征的 DataFrame，并存储在列表中。

最终输出是一个列表，其中每个元素对应一个商店的假期特征表。

In [32]:
def holiday_list(df_stores):

    listofseries = []
    
    for i in range(0,len(df_stores)):        
            df_holiday_dummies = pd.DataFrame(columns=['date'])
            df_holiday_dummies["date"] = df_holidays_events["date"]
    
            df_holiday_dummies["national_holiday"] = np.where(((df_holidays_events["type"] == "Holiday") & (df_holidays_events["locale"] == "National")), 1, 0)

            df_holiday_dummies["earthquake_relief"] = np.where(df_holidays_events['description'].str.contains('Terremoto Manabi'), 1, 0)

            df_holiday_dummies["christmas"] = np.where(df_holidays_events['description'].str.contains('Navidad'), 1, 0)

            df_holiday_dummies["football_event"] = np.where(df_holidays_events['description'].str.contains('futbol'), 1, 0)

            df_holiday_dummies["national_event"] = np.where(((df_holidays_events["type"] == "Event") & (df_holidays_events["locale"] == "National") & (~df_holidays_events['description'].str.contains('Terremoto Manabi')) & (~df_holidays_events['description'].str.contains('futbol'))), 1, 0)

            df_holiday_dummies["work_day"] = np.where((df_holidays_events["type"] == "Work Day"), 1, 0)

            df_holiday_dummies["local_holiday"] = np.where(((df_holidays_events["type"] == "Holiday") & ((df_holidays_events["locale_name"] == df_stores['state'][i]) | (df_holidays_events["locale_name"] == df_stores['city'][i]))), 1, 0)
                     
            listofseries.append(df_holiday_dummies)

    return listofseries

## 移除无意义数据：
删除没有任何假期特征标记的日期，减少数据体积并提高分析效率。

## 合并重复记录：
同一天内可能存在多个假期事件记录，通过分组取最大值的方法合并这些记录，确保数据一致性。

## 输出：
一个清洗后的假期特征列表，准备用于后续分析或建模。

In [33]:
def remove_0_and_duplicates(holiday_list):

    listofseries = []
    
    for i in range(0,len(holiday_list)):         
            df_holiday_per_store = list_of_holidays_per_store[i].set_index('date')

            df_holiday_per_store = df_holiday_per_store.loc[~(df_holiday_per_store==0).all(axis=1)]
            
            df_holiday_per_store = df_holiday_per_store.groupby('date').agg({'national_holiday':'max', 'earthquake_relief':'max', 
                                   'christmas':'max', 'football_event':'max', 
                                   'national_event':'max', 'work_day':'max', 
                                   'local_holiday':'max'}).reset_index()

            listofseries.append(df_holiday_per_store)

    return listofseries

### 输入：
假期特征的 DataFrame 列表，每个 DataFrame 包含一个商店的假期特征数据。

处理流程：
将假期特征转换为每日频率的时间序列。

填补缺失日期，缺失值填充为 0。

限制时间范围为 2013-01-01 至 2017-08-31。

将数据类型统一为 float32。

### 输出：
一个包含 54 个时间序列对象的列表，每个商店对应一个时间序列。

In [34]:
def holiday_TS_list_54(holiday_list):
    listofseries = []
    
    for i in range(0,54):
            holidays_TS = TimeSeries.from_dataframe(list_of_holidays_per_store[i], 
                                        time_col = 'date',
                                        fill_missing_dates=True,
                                        fillna_value=0,
                                        freq='D')
            
            holidays_TS = holidays_TS.slice(pd.Timestamp('20130101'),pd.Timestamp('20170831'))
            holidays_TS = holidays_TS.astype(np.float32)
            listofseries.append(holidays_TS)

    return listofseries

### 输入：

假期特征的原始数据和商店信息。

### 处理流程：

清洗假期数据，去除无效记录并合并重复日期。

转换为时间序列，统一时间范围并填补缺失值。

应用流水线，填补残余缺失值并标准化特征值。
### 输出：

标准化的时间序列对象列表，供后续模型使用。

In [35]:
list_of_holidays_per_store = holiday_list(df_stores)
list_of_holidays_per_store = remove_0_and_duplicates(list_of_holidays_per_store)   
list_of_holidays_store = holiday_TS_list_54(list_of_holidays_per_store)

holidays_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Filler")
holidays_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")

holidays_pipeline = Pipeline([holidays_filler, holidays_scaler])
holidays_transformed = holidays_pipeline.fit_transform(list_of_holidays_store)

In [36]:
display(len(holidays_transformed))
display(holidays_transformed[0].components.values)
display(holidays_transformed[0][:100])

54

array(['national_holiday', 'earthquake_relief', 'christmas',
       'football_event', 'national_event', 'work_day', 'local_holiday'],
      dtype=object)

<TimeSeries (DataArray) (date: 100, component: 7, sample: 1)> Size: 3kB
array([[[1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        [0.],
...
        [0.],
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]]], dtype=float32)
Coordinates:
  * date       (date) datetime64[ns] 800B 2013-01-01 2013-01-02 ... 2013-04-10
  * component  (component) object 56B 'national_holiday' ... 'local_holiday'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

### Promotion 

In [37]:
from tqdm import tqdm

### 输入：
训练集和测试集的促销数据（包含商店编号、商品类别、日期、促销商品数量等）。

商品类别列表 family_list。

### 处理流程：
合并训练集与测试集。

按商品类别筛选数据。

对每个商品类别的数据按商店编号分组，生成每日频率的时间序列。

转换时间序列值为浮点数，并存入字典。
### 输出：
包含所有商品类别及其促销时间序列的字典 family_promotion_dict。

In [38]:
df_promotion = pd.concat([df_train, df_test], axis=0)
df_promotion = df_promotion.sort_values(["store_nbr","family","date"])
df_promotion.tail()

family_promotion_dict = {}

for family in tqdm(family_list):
    df_family = df_promotion.loc[df_promotion['family'] == family]

    list_of_TS_promo = TimeSeries.from_group_dataframe(
                                df_family,
                                time_col="date",
                                group_cols=["store_nbr","family"],
                                value_cols="onpromotion",
                                fill_missing_dates=True,
                                freq='D')

    for ts in list_of_TS_promo:
        ts = ts.astype(np.float32)

    family_promotion_dict[family] = list_of_TS_promo

100%|██████████| 33/33 [00:16<00:00,  2.02it/s]


In [39]:
display(family_promotion_dict['AUTOMOTIVE'][53])

<TimeSeries (DataArray) (date: 1704, component: 1, sample: 1)> Size: 14kB
array([[[0.]],

       [[0.]],

       [[0.]],

       ...,

       [[0.]],

       [[0.]],

       [[0.]]])
Coordinates:
  * date       (date) datetime64[ns] 14kB 2013-01-01 2013-01-02 ... 2017-08-31
  * component  (component) object 8B 'onpromotion'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  static_covariates  store_nbr      family\ncomponent  ...
    hierarchy:          None

### 输入：
字典 family_promotion_dict，包含每个商品类别的促销时间序列列表。

### 处理流程：
使用流水线填补缺失值并标准化时间序列。

为每个时间序列计算 7 天和 28 天的移动平均特征。

合并原始时间序列与移动平均特征。

### 输出：
字典 promotion_transformed_dict，键为商品类别，值为处理后的促销时间序列特征列表。

In [40]:
promotion_transformed_dict = {}

for key in tqdm(family_promotion_dict):
    promo_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Fill NAs")
    promo_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaling")

    promo_pipeline = Pipeline([promo_filler,
                             promo_scaler])

    promotion_transformed = promo_pipeline.fit_transform(family_promotion_dict[key])

    # Moving Averages for Promotion Family Dictionaries
    promo_moving_average_7 = MovingAverageFilter(window=7)
    promo_moving_average_28 = MovingAverageFilter(window=28)

    promotion_covs = []

    for ts in promotion_transformed:
        ma_7 = promo_moving_average_7.filter(ts)
        ma_7 = TimeSeries.from_series(ma_7.pd_series())  
        ma_7 = ma_7.astype(np.float32)
        ma_7 = ma_7.with_columns_renamed(col_names=ma_7.components, col_names_new="promotion_ma_7")
        ma_28 = promo_moving_average_28.filter(ts)
        ma_28 = TimeSeries.from_series(ma_28.pd_series())  
        ma_28 = ma_28.astype(np.float32)
        ma_28 = ma_28.with_columns_renamed(col_names=ma_28.components, col_names_new="promotion_ma_28")
        promo_and_mas = ts.stack(ma_7).stack(ma_28)
        promotion_covs.append(promo_and_mas)

    promotion_transformed_dict[key] = promotion_covs

100%|██████████| 33/33 [00:39<00:00,  1.18s/it]


In [41]:
display(promotion_transformed_dict['AUTOMOTIVE'][0].components.values)
display(promotion_transformed_dict['AUTOMOTIVE'][53])

array(['onpromotion', 'promotion_ma_7', 'promotion_ma_28'], dtype=object)

<TimeSeries (DataArray) (date: 1704, component: 3, sample: 1)> Size: 41kB
array([[[0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.]],

       ...,

       [[0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.]]])
Coordinates:
  * date       (date) datetime64[ns] 14kB 2013-01-01 2013-01-02 ... 2017-08-31
  * component  (component) object 24B 'onpromotion' ... 'promotion_ma_28'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  static_covariates  store_nbr      family\nglobal_comp...
    hierarchy:          None

### Grouping the covariates

通过将时间特征、油价和油价的移动平均特征堆叠在一起，创建一个新的时间序列集合 general_covariates

In [42]:
general_covariates = time_cov_transformed.stack(oil_transformed).stack(oil_moving_averages)

In [43]:
display(general_covariates[:20])

<TimeSeries (DataArray) (time: 20, component: 10, sample: 1)> Size: 800B
array([[[0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [1.6666667e-01],
        [0.0000000e+00],
        [0.0000000e+00],
        [7.9296458e-01],
        [7.9240197e-01],
        [7.9615402e-01]],

       [[0.0000000e+00],
        [0.0000000e+00],
        [3.3333335e-02],
        [2.7397261e-03],
        [3.3333334e-01],
        [0.0000000e+00],
        [5.9276825e-04],
        [7.9296458e-01],
        [7.9253030e-01],
...
        [0.0000000e+00],
        [6.0000002e-01],
        [4.9315069e-02],
        [8.3333337e-01],
        [3.8461544e-02],
        [1.0669828e-02],
        [8.2364082e-01],
        [8.2179654e-01],
        [8.1540072e-01]],

       [[0.0000000e+00],
        [0.0000000e+00],
        [6.3333333e-01],
        [5.2054793e-02],
        [1.0000000e+00],
        [3.8461544e-02],
        [1.1262597e-02],
        [8.2506216e-01],
        [8.2311636e-01],
        [8.1704903e-01]]], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 160B 2013-01-01 2013-01-02 ... 2013-01-20
  * component  (component) object 80B 'year' 'month' ... 'oil_ma_7' 'oil_ma_28'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

将每个商店的节假日信息和其他时间特征（如油价、时间等）结合在一起，生成了一个包含所有商店的堆叠协变量数据集

In [44]:
store_covariates_future = []

for store in range(0,len(store_list)):
    stacked_covariates = holidays_transformed[store].stack(general_covariates)  
    store_covariates_future.append(stacked_covariates)

In [45]:
display(store_covariates_future[53][:2])

<TimeSeries (DataArray) (date: 2, component: 17, sample: 1)> Size: 136B
array([[[1.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [1.6666667e-01],
        [0.0000000e+00],
        [0.0000000e+00],
        [7.9296458e-01],
        [7.9240197e-01],
        [7.9615402e-01]],

       [[0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [3.3333335e-02],
        [2.7397261e-03],
        [3.3333334e-01],
        [0.0000000e+00],
        [5.9276825e-04],
        [7.9296458e-01],
        [7.9253030e-01],
        [7.9603612e-01]]], dtype=float32)
Coordinates:
  * date       (date) datetime64[ns] 16B 2013-01-01 2013-01-02
  * component  (component) object 136B 'national_holiday' ... 'oil_ma_28'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

### 未来协变量（促销（原本，7天滑窗，28天滑窗），节假日以及重要事件，时间（年月日，第几天，第几周，与时间相关的线性增长特征），油价（原本，7天滑窗，28天滑窗））
将每个商店的促销时间序列与其对应的未来协变量（如节假日、油价、时间等）堆叠起来，形成一个新的时间序列集合。这个集合 future_covariates_dict 将包含每个促销家庭下所有商店的联合协变量信息

In [46]:
future_covariates_dict = {}

for key in tqdm(promotion_transformed_dict):
    promotion_family = promotion_transformed_dict[key]
    covariates_future = [promotion_family[i].stack(store_covariates_future[i]) for i in range(0,len(promotion_family))]
    future_covariates_dict[key] = covariates_future

100%|██████████| 33/33 [00:03<00:00,  9.62it/s]


In [47]:
display(future_covariates_dict['AUTOMOTIVE'][0].components)    ## [商品种类][商店编号][时间序列]

Index(['onpromotion', 'promotion_ma_7', 'promotion_ma_28', 'national_holiday',
       'earthquake_relief', 'christmas', 'football_event', 'national_event',
       'work_day', 'local_holiday', 'year', 'month', 'day', 'dayofyear',
       'dayofweek', 'weekofyear', 'linear_increase', 'dcoilwtico', 'oil_ma_7',
       'oil_ma_28'],
      dtype='object', name='component')

In [48]:
display(future_covariates_dict['AUTOMOTIVE'][0][:2])

<TimeSeries (DataArray) (date: 2, component: 20, sample: 1)> Size: 320B
array([[[0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00],
        [1.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00],
        [1.66666672e-01],
        [0.00000000e+00],
        [0.00000000e+00],
        [7.92964578e-01],
        [7.92401969e-01],
        [7.96154022e-01]],
...
       [[0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00],
        [3.33333351e-02],
        [2.73972610e-03],
        [3.33333343e-01],
        [0.00000000e+00],
        [5.92768250e-04],
        [7.92964578e-01],
        [7.92530298e-01],
        [7.96036124e-01]]])
Coordinates:
  * date       (date) datetime64[ns] 16B 2013-01-01 2013-01-02
  * component  (component) object 160B 'onpromotion' ... 'oil_ma_28'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  static_covariates  store_nbr      family\nglobal_comp...
    hierarchy:          None

### Transactions – Past Covariates (过去协变量)

处理交易数据：将每个商店的交易数据转换为时间序列对象。

填补缺失数据：如果某些商店的交易数据在特定日期缺失，通过添加零填充的时间序列来弥补。

数据标准化：使用 MissingValuesFiller 填充缺失值，使用 Scaler 对数据进行标准化，使数据适合模型训练。

最终，transactions_transformed 中存储了处理后的、填补缺失值并进行缩放的交易数据，可以用于后续的模型训练和预测。

In [49]:
df_transactions.sort_values(["store_nbr","date"], inplace=True)

TS_transactions_list = TimeSeries.from_group_dataframe(
                                df_transactions,
                                time_col="date",
                                group_cols=["store_nbr"],
                                value_cols="transactions",
                                fill_missing_dates=True,
                                freq='D')

transactions_list = []

for ts in TS_transactions_list:
            series = TimeSeries.from_series(ts.pd_series())
            series = series.astype(np.float32)
            transactions_list.append(series)

transactions_list[24] = transactions_list[24].slice(start_ts=pd.Timestamp('20130102'), end_ts=pd.Timestamp('20170815'))

from datetime import datetime, timedelta

transactions_list_full = []

for ts in transactions_list:
    if ts.start_time() > pd.Timestamp('20130101'):
        end_time = (ts.start_time() - timedelta(days=1))
        delta = end_time - pd.Timestamp('20130101')
        zero_series = TimeSeries.from_times_and_values(
                                  times=pd.date_range(start=pd.Timestamp('20130101'), 
                                  end=end_time, freq="D"),
                                  values=np.zeros(delta.days+1))
        ts = zero_series.append(ts)
        ts = ts.with_columns_renamed(col_names=ts.components, col_names_new="transactions")
        transactions_list_full.append(ts)

transactions_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Filler")
transactions_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")

transactions_pipeline = Pipeline([transactions_filler, transactions_scaler])
transactions_transformed = transactions_pipeline.fit_transform(transactions_list_full)

In [ ]:
display(transactions_transformed[53][:2])    # [商店编号][时间序列]

<TimeSeries (DataArray) (time: 2, component: 1, sample: 1)> Size: 16B
array([[[0.        ]],

       [[0.55107675]]])
Coordinates:
  * time       (time) datetime64[ns] 16B 2013-01-01 2013-01-02
  * component  (component) object 8B 'transactions'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

### Create dataframe with id and reduce memory usage（历史协变量/滞后特征）

将 df_train 和 df_test 合并为一个统一的 DataFrame。

删除不需要的 onpromotion 列。

对数据按商店编号和产品类别进行排序。

将日期列转换为 datetime 类型，方便后续的时间处理。

查看合并后 DataFrame 的维度（行数和列数）。

In [51]:
df_indexes = pd.concat([df_train, df_test])
df_indexes = df_indexes.drop(['onpromotion'], axis=1)
df_indexes = df_indexes.sort_values(by=['store_nbr','family',])
df_indexes.date = pd.to_datetime(df_indexes.date)
df_indexes.shape

(3029400, 5)

In [52]:
df_indexes = df_indexes.set_index('date')
df_indexes.head()

,id,store_nbr,family,sales
date,,,,
2013-01-01,0,1,AUTOMOTIVE,0.0
2013-01-02,1782,1,AUTOMOTIVE,2.0
2013-01-03,3564,1,AUTOMOTIVE,3.0
2013-01-04,5346,1,AUTOMOTIVE,3.0
2013-01-05,7128,1,AUTOMOTIVE,5.0


通过循环对每个商店和产品类别的数据进行填充，确保所有商店、产品类别的销售数据都有完整的日期序列。如果某些日期没有销售数据，则填充缺失值（NaN）。这个过程为后续的时间序列分析或模型训练准备了一个完整且一致的时间索引

In [53]:
date_range = pd.date_range(start=df_indexes.index.min(), end=df_indexes.index.max(), freq="D")
df_indexes_filled = pd.DataFrame(columns=df_indexes.columns)

for family in tqdm(family_list):
    for store in store_list:
        temp_df = df_indexes.iloc[np.where((df_indexes.family == family)&(df_indexes.store_nbr == store))]
        temp_df = temp_df.reindex(date_range).fillna({'id': np.nan, 'store_nbr': store, 'family':family, 'sales': np.nan})
        df_indexes_filled = pd.concat([df_indexes_filled, temp_df])

df_indexes_filled.head()

100%|██████████| 33/33 [04:32<00:00,  8.27s/it]


,id,store_nbr,family,sales
2013-01-01,0.0,1.0,AUTOMOTIVE,0.0
2013-01-02,1782.0,1.0,AUTOMOTIVE,2.0
2013-01-03,3564.0,1.0,AUTOMOTIVE,3.0
2013-01-04,5346.0,1.0,AUTOMOTIVE,3.0
2013-01-05,7128.0,1.0,AUTOMOTIVE,5.0


给 df_indexes_filled 的索引命名为 'date'，使得索引更加清晰。

重置索引，以便后续操作使用默认的整数索引，并将日期列转为普通列。

按照商店编号和产品类别对数据进行排序，以便后续处理和分析。

In [54]:
df_indexes_filled.index.name = 'date'
df_indexes_filled = df_indexes_filled.reset_index()
df_indexes_filled = df_indexes_filled.sort_values(['store_nbr', 'family'])
df_indexes_filled.head()

,date,id,store_nbr,family,sales
0,2013-01-01,0.0,1.0,AUTOMOTIVE,0.0
1,2013-01-02,1782.0,1.0,AUTOMOTIVE,2.0
2,2013-01-03,3564.0,1.0,AUTOMOTIVE,3.0
3,2013-01-04,5346.0,1.0,AUTOMOTIVE,3.0
4,2013-01-05,7128.0,1.0,AUTOMOTIVE,5.0


In [55]:
last_train_date = pd.to_datetime(df_train.date.max())

In [56]:
import gc

In [57]:
del(df_train)
del(df_test)
del(df_stores)
del(df_holidays_events)
del(df_oil)
del(df_transactions)
del(df_indexes)
del(train_merged)

gc.collect()

157

# Model

In [58]:
from darts.models import LightGBMModel

We will use a general predictions function for blending and stacking.

val_df_size：用于设置验证集的天数大小（例如，验证集为最后几天的数据）。这个参数对于 stacking 技术非常重要，因为它影响到训练集和验证集的划分。在 blending 中，这个参数默认为0。

In [59]:
'''
The function takes two arguments as input:
model_params: model hyperparameters, tuning them can improve prediction accuracy.
val_df_size: number of days in the validation set. The parameter is needed to determine
the size of the validation sample in the stacking. For blending, set the default value to zero.
'''

def lgbm_predictions(model_params, val_df_size = 0):
    l_train_date = last_train_date - np.timedelta64(val_df_size, 'D')
    local_df_indexes = df_indexes_filled.iloc[np.where(df_indexes_filled.date > l_train_date)]
    
    submission_kaggle_list = []    
    cnt = 1
    
    for params in model_params:
        LGBM_Models_Submission = {}
        display("Training...")
            
        # Fit Model
        print(f'Start fit model {cnt}')
        for family in tqdm(family_list):        
            sales_family = family_TS_transformed_dict[family]
            # training_data: represents the number of sales in the training sample minus the sales for the val
            training_data = [ts[:1688-val_df_size] for ts in sales_family]
            # TCN_covariates: represents the future covariates associated with the target product family
            TCN_covariates = future_covariates_dict[family]
            # train_sliced: represents the number of sales associated with the target product family.
            # slice_intersect: function that you can see used simply ensures that the components span the same time interval. 
            # In the case of different time intervals an error message will appear if we try to combine them.
            train_sliced = [training_data[i].slice_intersect(TCN_covariates[i]) for i in range(0,len(training_data))]
            

            LGBM_Model_Submission = LightGBMModel(lags = params["lags"],
                                                  lags_future_covariates = params["lags_future_covariates"],
                                                  lags_past_covariates = params["lags_past_covariates"],
                                                  output_chunk_length=1,
                                                  random_state=2022,
                                                  gpu_use_dp= "false")


            LGBM_Model_Submission.fit(series=train_sliced, 
                                  future_covariates=TCN_covariates,
                                  # transactions_transformed: the past covariates do not need to be indexed on the target 
                                  # family because there is only one global `TimeSeries` per store.
                                  past_covariates=transactions_transformed)

            LGBM_Models_Submission[family] = LGBM_Model_Submission

        display("Predictions...")
        LGBM_Forecasts_Families_Submission = {}

        # Predict
        print(f'Start predict model {cnt}')
        for family in tqdm(family_list):
            sales_family = family_TS_transformed_dict[family]
            training_data = [ts[:1688-val_df_size] for ts in sales_family]
            LGBM_covariates = future_covariates_dict[family]
            train_sliced = [training_data[i].slice_intersect(TCN_covariates[i]) for i in range(0,len(training_data))]

            forecast_LGBM = LGBM_Models_Submission[family].predict(
                                                  n= 16 + val_df_size,
                                                  series=train_sliced,
                                                  future_covariates=LGBM_covariates,
                                                  past_covariates=transactions_transformed
                                                 )

            LGBM_Forecasts_Families_Submission[family] = forecast_LGBM

        # Transform Back
        print(f'Start transform Back {cnt}')
        LGBM_Forecasts_Families_back_Submission = {}

        for family in tqdm(family_list):
            LGBM_Forecasts_Families_back_Submission[family] = family_pipeline_dict[family].inverse_transform(LGBM_Forecasts_Families_Submission[family], partial=True)

        # Prepare Submission in Correct Format
        print(f'Start Prepare Submission {cnt}')
        for family in tqdm(LGBM_Forecasts_Families_back_Submission):
            for n in range(0,len(LGBM_Forecasts_Families_back_Submission[family])):
                if (family_TS_dict[family][n].univariate_values()[-21:] == 0).all():
                    LGBM_Forecasts_Families_back_Submission[family][n] = LGBM_Forecasts_Families_back_Submission[family][n].map(lambda x: x * 0)

        listofseries = []

        for store in tqdm(range(0,54)):
            for family in family_list:
                oneforecast = LGBM_Forecasts_Families_back_Submission[family][store].pd_dataframe()
                oneforecast.columns = ['y_pred']
                listofseries.append(oneforecast)

        df_forecasts = pd.concat(listofseries) 
        df_forecasts.reset_index(drop=True, inplace=True)

        # No Negative Forecasts
        print(f'Start No Negative Forecasts {cnt}')
        df_forecasts[df_forecasts < 0] = 0
        forecasts_kaggle = pd.concat([local_df_indexes['id'], df_forecasts.set_index(local_df_indexes.index)], axis=1)
        forecasts_kaggle = forecasts_kaggle.reset_index(drop=True)

        # Submission
        print(f'Start Submission {cnt}')
        submission_kaggle_list.append(forecasts_kaggle)
        cnt += 1
    
    return submission_kaggle_list, local_df_indexes

**Blending 中的预测结果**：

- 在 **Blending** 中，`lgbm_predictions` 函数训练多个模型，并对所有模型进行预测。每个模型的预测结果将存储在 `submission_kaggle_list` 中。
- 这些预测结果会被简单地进行 **平均** 或 **加权平均**，然后生成最终的预测结果。Blending 只是对多个模型的输出进行简单融合，因此不涉及训练元模型。

**Stacking 中的预测结果**：

- 在 **Stacking** 中，`lgbm_predictions` 函数同样用于训练多个 LGBM 模型，并生成每个模型的预测结果（存储在 `submission_kaggle_list` 中）。但在 Stacking 中，训练集的预测结果会被用作 **元模型** 的输入。
- 也就是说，Stacking 中的预测结果不仅仅是对每个模型输出结果的简单平均，而是会进一步用于训练一个新的模型（元模型），该元模型会学习如何结合这些基学习器的输出，从而生成更好的最终预测结果。

# Blending
The idea of blending is extremely simple and intuitive, which does not prevent the technique from showing good results.

Suppose if one model makes an error in one direction, then another model can correct this error if its predictions are biased in the other direction.

As a result, the mean value of the predictions may be closer to the true value than the predictions of individual models.

However, it should be noted that averaging the results of models can only be effective if the models are diverse and independent of each other.

If the models are highly correlated or have similar errors, averaging may not provide a meaningful improvement.

Let's set significantly different model hyperparameters to reduce the chance of error correlation:

In [60]:
model_params = [
    {"lags" : 63, "lags_future_covariates" : (14,1), "lags_past_covariates" : [-16,-17,-18,-19,-20,-21,-22]},
    {"lags" : 7, "lags_future_covariates" : (16,1), "lags_past_covariates" : [-16,-17,-18,-19,-20,-21,-22]},  
    {"lags" : 31, "lags_future_covariates" : (14,1), "lags_past_covariates" : [-16,-17,-18,-19,-20,-21,-22]},
    {"lags" : 365, "lags_future_covariates" : (14,1), "lags_past_covariates" : [-16,-17,-18,-19,-20,-21,-22]}, 
    {"lags" : 730, "lags_future_covariates" : (14,1), "lags_past_covariates" : [-16,-17,-18,-19,-20,-21,-22]}, 
    {"lags" : 1095, "lags_future_covariates" : (14,1), "lags_past_covariates" : [-16,-17,-18,-19,-20,-21,-22]}
]

In [61]:
submission_kaggle_list, clipped_indexes = lgbm_predictions(model_params)

'Training...'

Start fit model 1


  0%|          | 0/33 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.058610 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 43479
[LightGBM] [Info] Number of data points in the train set: 87750, number of used features: 375
[LightGBM] [Info] Start training from score 0.461126


  3%|▎         | 1/33 [00:03<01:46,  3.32s/it]

...

Start transform Back 6


100%|██████████| 33/33 [00:14<00:00,  2.32it/s]


Start Prepare Submission 6


100%|██████████| 54/54 [00:00<00:00, 114.00it/s]


Start No Negative Forecasts 6
Start Submission 6


We average the obtained predictions:

这段代码的作用是对多个模型的预测结果进行 **平均**，以减少单一模型预测误差的影响，并生成最终的销售预测（`sales`）。步骤包括：

- **合并** 各个模型的预测结果。
- **计算每个样本的平均预测值**。
- 最终输出一个包含 `id` 和 `sales` 的 DataFrame，准备提交预测结果。

In [62]:
submissions = submission_kaggle_list[0].copy()
submissions = submissions.rename(columns={'y_pred': 'y_pred_0'})

if len(submission_kaggle_list) > 1:
    for i in range(1, len(submission_kaggle_list)):
        y_pred = submission_kaggle_list[i]
        y_pred = y_pred.rename(columns={'y_pred': f'y_pred_{i}'})
        submissions = pd.concat([submissions, y_pred.drop(['id'], axis=1)], axis=1)

submissions['sales'] = submissions.loc[:, submissions.columns!='id'].mean(axis=1)
submissions.head()

,id,y_pred_0,y_pred_1,y_pred_2,y_pred_3,y_pred_4,y_pred_5,sales
0,3000888.0,3.466254,3.328092,3.507580,3.068898,3.731862,4.001414,3.517350
1,3002670.0,2.890935,2.603473,3.216886,3.253258,3.681850,4.065727,3.285355
2,3004452.0,3.972446,2.813861,4.004264,3.852360,3.615720,2.842600,3.516875
3,3006234.0,5.056782,3.124398,5.022578,4.502551,5.044515,4.717491,4.578052
4,3008016.0,1.801655,1.053930,1.463690,1.787588,2.311553,2.026005,1.740737


In [63]:
submission = submissions[['id', 'sales']]
submission = submission.sort_values('id')
submission.id = submission.id.astype('int32')
submission.head()

,id,sales
0,3000888,3.517350
16,3000889,0.000000
32,3000890,4.337862
48,3000891,2288.981609
64,3000892,0.031684


In [64]:
submission.to_csv('submission.csv', index=False)

# Stacking

Stacking is also an ensemble learning method, but it is based on a slightly different approach.
The idea sounds like this - let's use the predictions of weak and maximally different models as predictors of the meta-model.

We will split the training dataset into training and validation data.
Darts models will be trained on the training data and make predictions for the validation and test data.

In [65]:
val_df_size = 100
submission_kaggle_list, clipped_indexes = lgbm_predictions(model_params, val_df_size)

'Training...'

Start fit model 1


  0%|          | 0/33 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.043098 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 43353
[LightGBM] [Info] Number of data points in the train set: 82350, number of used features: 375
[LightGBM] [Info] Start training from score 0.456334


  3%|▎         | 1/33 [00:03<01:40,  3.15s/it]

...

Start transform Back 6


100%|██████████| 33/33 [00:14<00:00,  2.33it/s]


Start Prepare Submission 6


100%|██████████| 54/54 [00:00<00:00, 117.21it/s]


Start No Negative Forecasts 6
Start Submission 6


In [66]:
submissions = submission_kaggle_list[0].copy()
submissions = submissions.rename(columns={'y_pred': 'y_pred_0'})

if len(submission_kaggle_list) > 1:
    for i in range(1, len(submission_kaggle_list)):
        y_pred = submission_kaggle_list[i]
        y_pred = y_pred.rename(columns={'y_pred': f'y_pred_{i}'})
        submissions = pd.concat([submissions, y_pred.drop(['id'], axis=1)], axis=1)

submissions.head()

,id,y_pred_0,y_pred_1,y_pred_2,y_pred_3,y_pred_4,y_pred_5
0,2822688.0,3.676266,3.417199,3.199446,3.579452,3.115974,3.065060
1,2824470.0,3.232229,3.578848,2.721255,3.442423,3.270835,2.882628
2,2826252.0,3.599256,3.569002,3.272049,3.206762,3.518412,2.981064
3,2828034.0,4.008501,3.567155,2.867772,3.685296,3.150041,3.790595
4,2829816.0,3.639679,2.998901,2.978183,3.132120,3.369821,3.219728


In [67]:
clipped_indexes.head()

,date,id,store_nbr,family,sales
1588,2017-05-08,2822688.0,1.0,AUTOMOTIVE,5.0
1589,2017-05-09,2824470.0,1.0,AUTOMOTIVE,2.0
1590,2017-05-10,2826252.0,1.0,AUTOMOTIVE,2.0
1591,2017-05-11,2828034.0,1.0,AUTOMOTIVE,4.0
1592,2017-05-12,2829816.0,1.0,AUTOMOTIVE,4.0


In [68]:
submissions = pd.concat([submissions, clipped_indexes[['date', 'sales']].reset_index(drop=True)], axis=1)
submissions.head()

,id,y_pred_0,y_pred_1,y_pred_2,y_pred_3,y_pred_4,y_pred_5,date,sales
0,2822688.0,3.676266,3.417199,3.199446,3.579452,3.115974,3.065060,2017-05-08,5.0
1,2824470.0,3.232229,3.578848,2.721255,3.442423,3.270835,2.882628,2017-05-09,2.0
2,2826252.0,3.599256,3.569002,3.272049,3.206762,3.518412,2.981064,2017-05-10,2.0
3,2828034.0,4.008501,3.567155,2.867772,3.685296,3.150041,3.790595,2017-05-11,4.0
4,2829816.0,3.639679,2.998901,2.978183,3.132120,3.369821,3.219728,2017-05-12,4.0


In [69]:
del(submission_kaggle_list)
del(df_indexes_filled)
del(clipped_indexes)

gc.collect()

1307

Retrieve future covariates data from the TimeSeries:

In [70]:
future_covariates_df = pd.DataFrame(columns=['store_nbr', 'family'] + list(future_covariates_dict['AUTOMOTIVE'][0].columns))

for store in tqdm(range(0, 54)):
    for family in list(family_TS_dict.keys()):
        fut_cov_temp = pd.DataFrame(future_covariates_dict[family][store][1688-val_df_size:].values(),
                                    index=future_covariates_dict[family][store].time_index[1688-val_df_size:],
                                    columns=future_covariates_dict[family][store].columns)
        fut_cov_temp['store_nbr'] = store + 1
        fut_cov_temp['family'] = family
        future_covariates_df = pd.concat([future_covariates_df, fut_cov_temp])
        
future_covariates_df.head()

100%|██████████| 54/54 [00:08<00:00,  6.09it/s]


,store_nbr,family,onpromotion,promotion_ma_7,promotion_ma_28,national_holiday,earthquake_relief,christmas,football_event,national_event,...,year,month,day,dayofyear,dayofweek,weekofyear,linear_increase,dcoilwtico,oil_ma_7,oil_ma_28
2017-05-08,1,AUTOMOTIVE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.363636,0.233333,0.347945,0.000000,0.346154,0.941316,0.240081,0.241925,0.261599
2017-05-09,1,AUTOMOTIVE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.363636,0.266667,0.350685,0.166667,0.346154,0.941909,0.232737,0.244632,0.262407
2017-05-10,1,AUTOMOTIVE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.363636,0.300000,0.353425,0.333333,0.346154,0.942502,0.249793,0.247791,0.263210
2017-05-11,1,AUTOMOTIVE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.363636,0.333333,0.356164,0.500000,0.346154,0.943094,0.256070,0.251400,0.263959
2017-05-12,1,AUTOMOTIVE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.363636,0.366667,0.358904,0.666667,0.346154,0.943687,0.256307,0.255461,0.263794


Enrich model predictions with future covariate data:

In [71]:
data_with_preds = pd.concat([submissions, future_covariates_df.reset_index(drop=True)], axis=1)
data_with_preds = data_with_preds.set_index('date')
data_with_preds.head()

,id,y_pred_0,y_pred_1,y_pred_2,y_pred_3,y_pred_4,y_pred_5,sales,store_nbr,family,...,year,month,day,dayofyear,dayofweek,weekofyear,linear_increase,dcoilwtico,oil_ma_7,oil_ma_28
date,,,,,,,,,,,,,,,,,,,,,
2017-05-08,2822688.0,3.676266,3.417199,3.199446,3.579452,3.115974,3.065060,5.0,1,AUTOMOTIVE,...,1.0,0.363636,0.233333,0.347945,0.000000,0.346154,0.941316,0.240081,0.241925,0.261599
2017-05-09,2824470.0,3.232229,3.578848,2.721255,3.442423,3.270835,2.882628,2.0,1,AUTOMOTIVE,...,1.0,0.363636,0.266667,0.350685,0.166667,0.346154,0.941909,0.232737,0.244632,0.262407
2017-05-10,2826252.0,3.599256,3.569002,3.272049,3.206762,3.518412,2.981064,2.0,1,AUTOMOTIVE,...,1.0,0.363636,0.300000,0.353425,0.333333,0.346154,0.942502,0.249793,0.247791,0.263210
2017-05-11,2828034.0,4.008501,3.567155,2.867772,3.685296,3.150041,3.790595,4.0,1,AUTOMOTIVE,...,1.0,0.363636,0.333333,0.356164,0.500000,0.346154,0.943094,0.256070,0.251400,0.263959
2017-05-12,2829816.0,3.639679,2.998901,2.978183,3.132120,3.369821,3.219728,4.0,1,AUTOMOTIVE,...,1.0,0.363636,0.366667,0.358904,0.666667,0.346154,0.943687,0.256307,0.255461,0.263794


In [72]:
data_with_preds.store_nbr = data_with_preds.store_nbr.astype('int32')

Let's create a validation and test dataset.

The meta-model will use the predictions as training parameters and will attempt to identify dependencies between the predictions and the original targets.

In [73]:
val = data_with_preds.iloc[np.where(data_with_preds.index < '2017-08-16')]
test = data_with_preds.iloc[np.where(data_with_preds.index >= '2017-08-16')]
print(val.index.min(), val.index.max())
print(test.index.min(), test.index.max())

2017-05-08 00:00:00 2017-08-15 00:00:00
2017-08-16 00:00:00 2017-08-31 00:00:00


In [74]:
from lightgbm import LGBMRegressor

In [75]:
feature_cols = list(val.columns.drop(['id', 'sales', 'family']))

In [76]:
result = pd.DataFrame(columns=['id', 'sales'])

for family in tqdm(family_list):
    temp_val = val.iloc[np.where(val.family.values == family)]
    temp_test = test.iloc[np.where(test.family.values == family)]

    temp_val['sales'] = np.log1p(temp_val['sales'])

    lgbm = LGBMRegressor()
    lgbm.fit(temp_val[feature_cols], temp_val['sales'])
    y_pred = lgbm.predict(temp_test[feature_cols])
    y_pred = np.expm1(y_pred)

    temp_test = pd.concat([temp_test['id'].reset_index(drop=True), pd.Series(y_pred, name='sales')], axis=1)
    result = pd.concat([result, temp_test])

result['sales'][result['sales'] < 0] = 0

  3%|▎         | 1/33 [00:00<00:04,  7.89it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000704 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2323
[LightGBM] [Info] Number of data points in the train set: 5400, number of used features: 21
[LightGBM] [Info] Start training from score 1.871664
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000615 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2152
[LightGBM] [Info] Number of data points in the train set: 5400, number of used features: 19
[LightGBM] [Info] Start training from score 0.103411


 12%|█▏        | 4/33 [00:00<00:02,  9.94it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000431 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2400
[LightGBM] [Info] Number of data points in the train set: 5400, number of used features: 21
[LightGBM] [Info] Start training from score 1.572546
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000349 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2915
[LightGBM] [Info] Number of data points in the train set: 5400, number of used features: 21
[LightGBM] [Info] Start training from score 7.967561
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000755 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1622
[LightGBM] [Info] Number of data points in the train set: 5400, number of used features: 18
[LightGBM] [Info] Start traini

 21%|██        | 7/33 [00:00<00:02, 10.18it/s]

...

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000380 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2745
[LightGBM] [Info] Number of data points in the train set: 5400, number of used features: 21
[LightGBM] [Info] Start training from score 5.574696
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000334 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2434
[LightGBM] [Info] Number of data points in the train set: 5400, number of used features: 21
[LightGBM] [Info] Start training from score 4.221888


 94%|█████████▍| 31/33 [00:02<00:00, 10.01it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000349 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2915
[LightGBM] [Info] Number of data points in the train set: 5400, number of used features: 21
[LightGBM] [Info] Start training from score 7.334575
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000898 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2526
[LightGBM] [Info] Number of data points in the train set: 5400, number of used features: 21
[LightGBM] [Info] Start training from score 0.835347


100%|██████████| 33/33 [00:03<00:00, 10.34it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000383 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2339
[LightGBM] [Info] Number of data points in the train set: 5400, number of used features: 21
[LightGBM] [Info] Start training from score 2.334235


In [77]:
submission_stack = result.sort_values('id')
submission_stack.id = submission_stack.id.astype('int32')
submission_stack.head()

,id,sales
0,3000888,3.750733
0,3000889,0.000000
0,3000890,5.155098
0,3000891,2268.181973
0,3000892,0.000000


In [78]:
submission_stack.to_csv('submission_stack.csv', index=False)